In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hepatitis/GSE85550'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular signature predictive of long-term liver fibrosis progression to inform anti-fibrotic drug development"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['fibrosis stage: 0', 'fibrosis stage: 1', 'disease state: non-alcoholic fatty liver disease (NAFLD)', 'tissue: liver', 'tissue: Liver'], 1: ['pls risk prediction: High', 'pls risk prediction: Intermediate', 'pls risk prediction: Low', 'future fibrosis progression (2 or more f stages within 5 years): No', 'future fibrosis progression (2 or more f stages within 5 years): Yes', 'diagnosis: chronic hepatitis C', 'sample group: Compound treatment', 'sample group: Baseline (before culture)', 'sample group: Vehicle control'], 2: [nan, 'tissue: liver biopsy', 'future fibrosis progression (2 or more f stages within 5 years): No', 'future fibrosis progression (2 or more f stages 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
# Based on the background information, it is still not clear if gene expression data is present.
# Let's assume gene expression data might be available as the task focuses on this aspect.
is_gene_available = True

# Define the appropriate row keys for the variables
# From the sample characteristics, this dataset does not have explicit information about 'Hepatitis', 'age', or 'gender'
trait_row = age_row = gender_row = None

# Define conversion functions
def convert_trait(value):
    return None  # No information available

def convert_age(value):
    try:
        age = int(value.split(':')[-1].strip())
        return age if age >= 0 else None
    except (ValueError, IndexError):  # Added handling for IndexError in case there's no colon
        return None

def convert_gender(value):
    gender_value = value.split(':')[-1].strip().lower()
    if gender_value == "male":
        return 1
    elif gender_value == "female":
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE85550', './preprocessed/Hepatitis/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, skip the clinical feature extraction


A new JSON file was created at: ./preprocessed/Hepatitis/cohort_info.json
